In [1]:
!pip install langchain langchain-core langchain-community pydantic duckduckgo-search langchain_experimental

INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-experimental to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 124.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the s

In [3]:
!pip install -U ddgs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 10.5 MB/s eta 0:00:00


# Built-in Tools


1) DuckDuckGo Search

In [4]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()
results = search.run("Current ipl 2026 retentions")
print(results)

3 hours ago - IPL 2026 Retention List Highlights: After all 10 teams released their released and retained lists for IPL 2026 ahead of the auctions, there were a few surprises with Chennai Super Kings releasing Matheesha Pathirana, Devon Conway and Rachin Ravindra, Kolkata letting go of players like Andre Russell, Quinton de Kock and Venkatesh Iyer and Lucknow Super Giants sending Ravi Bishnoi into the auction pool. 3 hours ago - The Shreyas Iyer-led side lost to Royal Challengers Bengaluru in the IPL 2025 final. PBKS haven't won an IPL title yet. ... CSK's retention list is here: Ruturaj Gaikwad (Captain), Ayush Mhatre, M.S. 16 hours ago - Earlier, charismatic all-rounder Ravindra Jadeja and keeper-batter Sanju Samson have been traded to Rajasthan Royals and Chennai Super Kings respectively. With the deadline for player retention ending later on Sunday, CSK stalwart Jadeja has switched base to Rajasthan Royals albeit at a much reduced fee, while Samson, who has led RR for four seasons,

2) Shell Tool

In [ ]:
from langchain_community.tools import ShellTool
shellTool = ShellTool()
results = shellTool.invoke('ls')
print(results)

# Custom Tools

1) Using @tool decorator

In [8]:
from langchain_core.tools import tool

In [9]:
# Step 1) Create a function
def multiply(a,b):
  """ Multiply two numbers"""
  return a*b

In [10]:
# Step 2) Add type hints
def multiply(a: int, b:int) -> int:
  """ Multiply two numbers"""
  return a*b

In [15]:
# Step 3) add tool decorator

@tool
def multiply(a: int, b: int) -> int:
  """ Multiply two numbers"""
  return a*b

In [16]:
result = multiply.invoke({"a": 2, "b": 3})
print(result)

6


In [18]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [19]:
multiply.args_schema.model_json_schema()

{'description': 'Multiply two numbers',
 'properties': {'a': {'title': 'A', 'type': 'integer'},
  'b': {'title': 'B', 'type': 'integer'}},
 'required': ['a', 'b'],
 'title': 'multiply',
 'type': 'object'}

2) Using Pydantic StructuredTool

In [20]:
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

In [21]:
class MultiplyIp(BaseModel):
  a : int = Field(required = True, description="first number to add")
  b : int = Field(required = True, description="second number to add")

In [22]:
def multiplyStructured(a: int, b: int) -> int:
  return a*b

In [24]:
multiplyTool = StructuredTool.from_function(
    func = multiplyStructured,
    name = "multiply",
    description = "Multiply two numbers",
    args_schema = MultiplyIp
)

In [25]:
result = multiplyTool.invoke({"a": 2, "b": 3})
print(result)
print(multiplyTool.name)
print(multiplyTool.description)
print(multiplyTool.args)

6
multiply
Multiply two numbers
{'a': {'description': 'first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


3) Using BaseTool Class

In [26]:
from langchain.tools import BaseTool
from typing import Type

In [27]:
class MultiplyInput(BaseModel):
  a : int = Field(required = True, description="First number to add")
  b : int = Field(required = True, description="Second number to add")

In [29]:
class MultiplyTool(BaseTool):
  name: str = "multiply"
  description: str = "Multiply two numbers"
  args_schema: Type[BaseModel] = MultiplyInput

  def _run(self, a: int, b:int) -> int:
    return a*b

In [32]:
# Object
mulTool = MultiplyTool()

In [33]:
result = mulTool.invoke({"a":3, "b": 3})
print(result)
print(mulTool.name)
print(mulTool.description)
print(mulTool.args)

9
multiply
Multiply two numbers
{'a': {'description': 'First number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'Second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


# Custom ToolKit

In [35]:
from langchain_core.tools import Tool

# Custom Tool 1
@tool
def add(a:int, b:int) -> int:
  """ Add two numbers"""
  return a+b

# Custom Tool 2
@tool
def multiply(a:int, b:int) ->int:
  """Multiply two Numbers"""
  return a*b

In [36]:
class MathToolKit:
  def getTools(self):
    return [add, multiply]


In [37]:
toolKit = MathToolKit()
tools = toolKit.getTools()

for tool in tools:
  print(tool.name, "-->", tool.description)

add --> Add two numbers
multiply --> Multiply two Numbers
